In [1]:
import pandas as pd
from langchain_chroma import Chroma

# Load Data

In [2]:
# movies = pd.read_csv('title.basics.tsv', sep='\t')
movies = pd.read_csv('dataset/imdb_top_1000.csv')
movies = movies.rename(columns={
    "Overview":"movies_description",
    "Series_Title":"movies_title"
})
movies.head()

,Poster_Link,movies_title,Released_Year,Certificate,Runtime,Genre,IMDB_Rating,movies_description,Meta_score,Director,Star1,Star2,Star3,Star4,No_of_Votes,Gross
0,https://m.media-amazon.com/images/M/MV5BMDFkYT...,The Shawshank Redemption,1994,A,142 min,Drama,9.3,Two imprisoned men bond over a number of years...,80.0,Frank Darabont,Tim Robbins,Morgan Freeman,Bob Gunton,William Sadler,2343110,"28,341,469"
1,https://m.media-amazon.com/images/M/MV5BM2MyNj...,The Godfather,1972,A,175 min,"Crime, Drama",9.2,An organized crime dynasty's aging patriarch t...,100.0,Francis Ford Coppola,Marlon Brando,Al Pacino,James Caan,Diane Keaton,1620367,"134,966,411"
2,https://m.media-amazon.com/images/M/MV5BMTMxNT...,The Dark Knight,2008,UA,152 min,"Action, Crime, Drama",9.0,When the menace known as the Joker wreaks havo...,84.0,Christopher Nolan,Christian Bale,Heath Ledger,Aaron Eckhart,Michael Caine,2303232,"534,858,444"
3,https://m.media-amazon.com/images/M/MV5BMWMwMG...,The Godfather: Part II,1974,A,202 min,"Crime, Drama",9.0,The early life and career of Vito Corleone in ...,90.0,Francis Ford Coppola,Al Pacino,Robert De Niro,Robert Duvall,Diane Keaton,1129952,"57,300,000"
4,https://m.media-amazon.com/images/M/MV5BMWU4N2...,12 Angry Men,1957,U,96 min,"Crime, Drama",9.0,A jury holdout attempts to prevent a miscarria...,96.0,Sidney Lumet,Henry Fonda,Lee J. Cobb,Martin Balsam,John Fiedler,689845,"4,360,000"


In [3]:
movies['page_content'] = "Title: " + movies['movies_title'] + "\n" + \
    "Genre: " + movies['Genre'] + "\n" + \
    "Description: " + movies['movies_description']
movies.head()

,Poster_Link,movies_title,Released_Year,Certificate,Runtime,Genre,IMDB_Rating,movies_description,Meta_score,Director,Star1,Star2,Star3,Star4,No_of_Votes,Gross,page_content
0,https://m.media-amazon.com/images/M/MV5BMDFkYT...,The Shawshank Redemption,1994,A,142 min,Drama,9.3,Two imprisoned men bond over a number of years...,80.0,Frank Darabont,Tim Robbins,Morgan Freeman,Bob Gunton,William Sadler,2343110,"28,341,469",Title: The Shawshank Redemption\nGenre: Drama\...
1,https://m.media-amazon.com/images/M/MV5BM2MyNj...,The Godfather,1972,A,175 min,"Crime, Drama",9.2,An organized crime dynasty's aging patriarch t...,100.0,Francis Ford Coppola,Marlon Brando,Al Pacino,James Caan,Diane Keaton,1620367,"134,966,411","Title: The Godfather\nGenre: Crime, Drama\nDes..."
2,https://m.media-amazon.com/images/M/MV5BMTMxNT...,The Dark Knight,2008,UA,152 min,"Action, Crime, Drama",9.0,When the menace known as the Joker wreaks havo...,84.0,Christopher Nolan,Christian Bale,Heath Ledger,Aaron Eckhart,Michael Caine,2303232,"534,858,444","Title: The Dark Knight\nGenre: Action, Crime, ..."
3,https://m.media-amazon.com/images/M/MV5BMWMwMG...,The Godfather: Part II,1974,A,202 min,"Crime, Drama",9.0,The early life and career of Vito Corleone in ...,90.0,Francis Ford Coppola,Al Pacino,Robert De Niro,Robert Duvall,Diane Keaton,1129952,"57,300,000","Title: The Godfather: Part II\nGenre: Crime, D..."
4,https://m.media-amazon.com/images/M/MV5BMWU4N2...,12 Angry Men,1957,U,96 min,"Crime, Drama",9.0,A jury holdout attempts to prevent a miscarria...,96.0,Sidney Lumet,Henry Fonda,Lee J. Cobb,Martin Balsam,John Fiedler,689845,"4,360,000","Title: 12 Angry Men\nGenre: Crime, Drama\nDesc..."


In [4]:
movies = movies[["page_content", "Poster_Link"]]

In [5]:
from langchain.document_loaders import DataFrameLoader

docs = DataFrameLoader(
    movies,
    page_content_column = "page_content"
).load()

In [6]:
docs[:3]

[Document(page_content='Title: The Shawshank Redemption\nGenre: Drama\nDescription: Two imprisoned men bond over a number of years, finding solace and eventual redemption through acts of common decency.', metadata={'Poster_Link': 'https://m.media-amazon.com/images/M/MV5BMDFkYTc0MGEtZmNhMC00ZDIzLWFmNTEtODM1ZmRlYWMwMWFmXkEyXkFqcGdeQXVyMTMxODk2OTU@._V1_UX67_CR0,0,67,98_AL_.jpg'}),
 Document(page_content="Title: The Godfather\nGenre: Crime, Drama\nDescription: An organized crime dynasty's aging patriarch transfers control of his clandestine empire to his reluctant son.", metadata={'Poster_Link': 'https://m.media-amazon.com/images/M/MV5BM2MyNjYxNmUtYTAwNi00MTYxLWJmNWYtYzZlODY3ZTk3OTFlXkEyXkFqcGdeQXVyNzkwMjQ5NzM@._V1_UY98_CR1,0,67,98_AL_.jpg'}),
 Document(page_content='Title: The Dark Knight\nGenre: Action, Crime, Drama\nDescription: When the menace known as the Joker wreaks havoc and chaos on the people of Gotham, Batman must accept one of the greatest psychological and physical tests of hi

# Calculate Cost Of Embeddings

In [7]:
import tiktoken

encoder = tiktoken.get_encoding("cl100k_base")

token_per_docs = [len(encoder.encode(doc.page_content)) for doc in docs]

total_tokens = sum(token_per_docs)

In [8]:
total_tokens

47059

cost per token is $0.0001 so ->

In [9]:
(total_tokens / 1000) * 0.0001

0.0047059

# Embeddings

## OpenAI

In [10]:
from langchain_openai import OpenAIEmbeddings
from dotenv import load_dotenv
import os

load_dotenv()
openai_api_key = os.getenv('OPENAI_API_KEY')

embeddings = OpenAIEmbeddings(openai_api_key=openai_api_key)

# Prepare VectorDB

In [11]:
persist_directory = "chroma_db"

# save ke local
db = Chroma.from_documents(
    documents=docs, embedding=embeddings, persist_directory=persist_directory
)

# load dari lokal
db = Chroma(persist_directory = persist_directory, embedding_function=embeddings)

# Test

In [12]:
retriever = db.as_retriever(search_type="similarity", search_kwargs={"k": 6})
retrieved_docs = retriever.invoke("What movies name that the story tells about an organized crime dynasty's?")
print(len(retrieved_docs))


query = "is there any move that is story about two imprisoned men bond over a number of years?"
docs = db.similarity_search(query)
print(docs)

6
[Document(page_content='Title: Miracle in cell NO.7\nGenre: Drama\nDescription: A story of love between a mentally-ill father who was wrongly accused of murder and his lovely six years old daughter. The prison would be their home. Based on the 2013 Korean movie 7-beon-bang-ui seon-mul (2013).', metadata={'Poster_Link': 'https://m.media-amazon.com/images/M/MV5BOGE3N2QxN2YtM2ZlNS00MWIyLWE1NDAtYWFlN2FiYjY1MjczXkEyXkFqcGdeQXVyOTUwNzc0ODc@._V1_UY98_CR1,0,67,98_AL_.jpg'}), Document(page_content='Title: Miracle in cell NO.7\nGenre: Drama\nDescription: A story of love between a mentally-ill father who was wrongly accused of murder and his lovely six years old daughter. The prison would be their home. Based on the 2013 Korean movie 7-beon-bang-ui seon-mul (2013).', metadata={'Poster_Link': 'https://m.media-amazon.com/images/M/MV5BOGE3N2QxN2YtM2ZlNS00MWIyLWE1NDAtYWFlN2FiYjY1MjczXkEyXkFqcGdeQXVyOTUwNzc0ODc@._V1_UY98_CR1,0,67,98_AL_.jpg'}), Document(page_content='Title: Miracle in cell NO.7\nGen

In [13]:
retrieved_docs[0].page_content

'Title: Once Upon a Time in America\nGenre: Crime, Drama\nDescription: A former Prohibition-era Jewish gangster returns to the Lower East Side of Manhattan over thirty years later, where he once again must confront the ghosts and regrets of his old life.'

In [14]:
from langchain_openai import ChatOpenAI
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain


def create_conversational_chain(vector_store):
    # Create llm
    llm = ChatOpenAI(model="gpt-3.5-turbo-0125")

    memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

    chain = ConversationalRetrievalChain.from_llm(llm=llm, chain_type='stuff',
                                                 retriever=vector_store.as_retriever(search_kwargs={"k": 2}),
                                                 memory=memory)
    return chain

In [15]:
def conversation_chat(query, chain):
    result = chain.invoke(
        {
            "question": query, 
            # "chat_history": history
        }
    )
    # history.append((query, result["answer"]))
    return result["answer"]

In [16]:
chain = create_conversational_chain(db)

In [17]:
output = conversation_chat(query, chain)

In [18]:
output

'Yes, the movie "The Shawshank Redemption" is a story about two imprisoned men who bond over a number of years.'